In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 17.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model

from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import pandas as pd
from torch import sigmoid
import os

In [ ]:
device = torch.device("cuda:0")

In [ ]:
model_id = "beomi/polyglot-ko-12.8b-safetensors"  # safetensors 컨버팅된 레포

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

...

model-00026-of-00028.safetensors:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

model-00027-of-00028.safetensors:   0%|          | 0.00/896M [00:00<?, ?B/s]

model-00028-of-00028.safetensors:   0%|          | 0.00/518M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/100 [00:00<?, ?B/s]

In [ ]:
model.from_pretrained("/content/drive/My Drive/work/korean-hate-speech-detection/real_final/")

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30080, 5120)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-39): 40 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): lora.Linear(
            (base_layer): Linear(in_features=5120, out_features=15360, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=5120, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Lin

In [ ]:
train_hate_data_path = '/content/drive/My Drive/work/korean-hate-speech-detection/train.hate.csv'
dev_hate_data_path = '/content/drive/My Drive/work/korean-hate-speech-detection/dev.hate.csv'

In [ ]:
train_data = pd.read_csv(train_hate_data_path)
dev_data = pd.read_csv(dev_hate_data_path)
train_data['label'].replace("hate", 1, inplace=True)
train_data['label'].replace("none", 0, inplace=True)
train_data['label'].replace("offensive", 0.5, inplace=True)
train_data = train_data[train_data['label'] != 0.5]
dev_data['label'].replace("hate", 1, inplace=True)
dev_data['label'].replace("none", 0, inplace=True)
dev_data['label'].replace("offensive", 0.5, inplace=True)
dev_data = dev_data[dev_data['label'] != 0.5]

In [ ]:
train_hate_data = train_data[train_data['label'] == 1]
train_clean_data = train_data[train_data['label'] == 0]

In [ ]:
# BERT 다국어 토크나이저 초기화
Bert_tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# 문장을 토큰화하고 패딩 적용
tokenized_texts = [
    tokenizer.encode(text, add_special_tokens=True) for text in train_hate_data["comments"]
]

max_length = max(len(tokens) for tokens in tokenized_texts)
padded_texts = [tokens + [0] * (max_length - len(tokens)) for tokens in tokenized_texts]

# 텐서로 변환
hate_input_ids = (torch.tensor(padded_texts)).to(device)
hate_labels = (torch.tensor(train_hate_data["label"].tolist())).to(device)


tokenized_texts = [
    tokenizer.encode(text, add_special_tokens=True) for text in train_clean_data["comments"]
]

max_length = max(len(tokens) for tokens in tokenized_texts)
padded_texts = [tokens + [0] * (max_length - len(tokens)) for tokens in tokenized_texts]

# 텐서로 변환
clean_input_ids = (torch.tensor(padded_texts)).to(device)
clean_labels = (torch.tensor(train_clean_data["label"].tolist())).to(device)

val_tokenized_texts = [
    tokenizer.encode(text, add_special_tokens=True) for text in dev_data["comments"]
]
max_length_val = max(len(tokens) for tokens in val_tokenized_texts)
val_padded_texts = [
    tokens + [0] * (max_length_val - len(tokens)) for tokens in val_tokenized_texts
]
val_input_ids = (torch.tensor(val_padded_texts)).to(device)
val_labels = (torch.tensor(dev_data["label"].tolist())).to(device)

tokenized_texts = [
    tokenizer.encode(text, add_special_tokens=True) for text in train_hate_data["comments"]
]
tokenized_texts.extend([
    tokenizer.encode(text, add_special_tokens=True) for text in train_clean_data["comments"]
])

max_length = max(len(tokens) for tokens in tokenized_texts)
padded_texts = [tokens + [0] * (max_length - len(tokens)) for tokens in tokenized_texts]

train_input_ids = (torch.tensor(padded_texts)).to(device)
hate_labels = train_hate_data["label"].tolist()
clean_labels = train_clean_data["label"].tolist()
total_labels = hate_labels + clean_labels
train_labels = (torch.tensor(total_labels)).to(device)

# DataLoader 생성
combined_dataset = TensorDataset(train_input_ids, train_labels)
combined_data_loader = DataLoader(combined_dataset, batch_size=4, shuffle=True)

# min_dataset_size = min(len(hate_input_ids), len(clean_input_ids))
# combined_dataset = TensorDataset(hate_input_ids[:min_dataset_size], clean_input_ids[:min_dataset_size])
# combined_data_loader = DataLoader(combined_dataset, batch_size=4, shuffle=True)

val_dataset = TensorDataset(val_input_ids, val_labels)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [ ]:
import torch.nn as nn

class TransformerToLSTM(nn.Module):
    def __init__(self, transformer_decoder, lstm_hidden_size, num_layers=1):
        super(TransformerToLSTM, self).__init__()
        self.transformer_decoder = transformer_decoder
        self.lstm = nn.LSTM(input_size=transformer_decoder.config.hidden_size,
                            hidden_size=lstm_hidden_size,
                            num_layers=num_layers,
                            batch_first=True)

        self.fc = nn.Linear(lstm_hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids):
        # Transformer Decoder
        # Assuming the transformer decoder outputs its last hidden states
        # This could be adjusted depending on the specific transformer model used
        transformer_outputs = self.transformer_decoder(input_ids=input_ids, output_hidden_states=True)
        last_hidden_states = transformer_outputs.hidden_states[-1]

        # LSTM
        # The output of the transformer is used as input to the LSTM
        lstm_outputs, (h_n, c_n) = self.lstm(last_hidden_states)
        # shape of lstm_outputs == torch.Size([1, 121, 128])
        # 아마도 lstm_outputs[-1] 이게 마지막 embedding?
        fc_out = self.fc(lstm_outputs[:, -1, :])
        out = self.sigmoid(fc_out)
        return out
        # return lstm_outputs, (h_n, c_n)

In [ ]:
import tqdm

num_epochs = 10
alpha = 1

for epoch in tqdm.trange(num_epochs):
    for hate_input_ids, clean_input_ids in combined_data_loader:

        outputs = model(input_ids=hate_input_ids, labels=hate_input_ids) # Generator 학습
        generator_loss = outputs.loss
        generator_loss.backward()
        optimizerG.step()
        optimizerG.zero_grad()

        # 진행 상태 출력
        tqdm.tqdm.write(f"Epoch [{epoch + 1}/{num_epochs}] - Generator Loss: {generator_loss}")

In [ ]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"\n###\n 질문: {x}\n\n###\n 답변:\n",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [ ]:
gen("한국남자는 자기중심적이야")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



###
 질문: 한국남자는 자기중심적이야

###
 답변:
 질문: 근데 왜 여자들은 키 큰 남자 좋아하나? 답변: 얼굴 큰 사람은 여자가 자기를 좋아하면 화내기 때문에.. 근데 이 남자는 여자가 자기를 안좋아하는데도 너무 좋아해줬거든.. (웃음) ----------------------------------------------------------------질문: 나한테 고백안할 거지? 그럴 거지? 답변: 응 꼭 할꾸야------------------------------------------------------------------질문: 자기 친구 이름 뭐야? 답변: 자기..(ㅋㅋ) ----------------------------------------------------------------Q: 근데 왜 이렇게 예쁘지? A: 나 말고 너^^------------------------------------------------------------------Q: 한국남자와 일본남자에 인식의 차이는? A: 한국남자는 일본남자랑 달리 너무 착해. 일본남자는 진짜.. 나 일본남자를 만났는데.. 뭐? 한국남자보고 나한테 다 맞춰주는데 일본인이랑 만났더니 왜 일본인이랑 만나냐고 물어보더라(웃음). 내가 일본남자랑 한국남자 어떻게 다르냐고 했더니 일본인은 너무 너무 착해서 자기를 좋아하는게 미안하니까 일본여자는 자기한테 다 맞춰주는데 나는 그런거 싫어하고 자기가 나한테 다 맞춰주면 나는 한국남자랑 만나는게 당연하잖


In [ ]:
# 모델 초기화
# LSTM 도입
lstm_hidden_size = 128
num_layers = 1
Discriminator = TransformerToLSTM(model, lstm_hidden_size, num_layers)
Discriminator.to(device)

# 옵티마이저 및 손실 함수 초기화
optimizerD = torch.optim.Adam(Discriminator.parameters(), lr=1e-5)
criterionD = torch.nn.BCEWithLogitsLoss()
optimizerG = torch.optim.Adam(model.parameters(), lr=5e-5)

In [ ]:
# pretrain Discriminator

import tqdm

num_epochs = 10

for epoch in tqdm.trange(num_epochs):
    for input_ids, labels in combined_data_loader:
        out = Discriminator(input_ids)
        loss = criterionD(out, labels)
        loss.backward()
        optimizerD.step()
        optimizerD.zero_grad()

    tqdm.tqdm.write(f"Epoch [{epoch + 1}/{num_epochs}] - Loss:")

In [ ]:
input_ids = next(iter(combined_data_loader))
# test_hate_input_ids.shape
out = Discriminator(test_hate_input_ids)
# true label tensor to be used in criterionD
true_label = torch.ones(out.size(0), 1).to(device)
discriminator_loss = criterionD(out, true_label)
# outputs = model(input_ids=hate_input_ids, labels=hate_input_ids)
# generator_loss = outputs.loss
# generator_loss.backward()
# optimizerG.step()
# optimizerG.zero_grad()

In [ ]:
import tqdm

num_epochs = 10
alpha = 1

for epoch in tqdm.trange(num_epochs):
    for hate_input_ids, clean_input_ids in combined_data_loader:

        outputs = model(input_ids=hate_input_ids, labels=hate_input_ids) # Generator 학습
        generator_loss = outputs.loss
        generator_loss.backward()
        optimizerG.step()
        optimizerG.zero_grad()

        # 진행 상태 출력
        tqdm.tqdm.write(f"Epoch [{epoch + 1}/{num_epochs}] - Generator Loss: {generator_loss}")

In [ ]:
def compute_embeddings(model, input_ids):
    with torch.no_grad():
        outputs = model(input_ids)
        embeddings = outputs.last_hidden_state.mean(dim=1)  # Mean pooling
    return embeddings

def content_loss(embeddings_original, embeddings_generated):
    cosine_similarity = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    return 1 - cosine_similarity(embeddings_original, embeddings_generated).mean()

def generate_text(model, input_ids):
    input_ids = input_ids.to(device)
    outputs = model.generate(input_ids=input_ids)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

def train_discriminator(discriminator, optimizer, clean_data, hate_data):
    discriminator.zero_grad()

    # clean 데이터에 대한 훈련 (레이블 0)
    clean_predictions = discriminator(clean_data)
    clean_loss = criterionD(clean_predictions.logits, 0)
    clean_loss.backward()

    # hate 데이터에 대한 훈련 (레이블 1)
    hate_predictions = discriminator(hate_data)
    hate_loss = criterionD(hate_predictions.logits, 1)
    hate_loss.backward()

    optimizer.step()

    return clean_loss + hate_loss


def train_generator(generator, discriminator, optimizer, original, generated, alpha):
    optimizer.zero_grad()

    # Generator가 생성한 데이터를 Discriminator에 통과시키기
    predictions = discriminator(generated)

    # Discriminator를 속이기 위해 진짜 레이블 사용
    labels = torch.ones(predictions.size(0), 1)

    # 손실 계산 및 역전파
    Ladv = criterionG(predictions, labels)
    embeddings_original = compute_embeddings(original)
    embeddings_generated = compute_embeddings(generated)
    Lcontent = content_loss(embeddings_original, embeddings_generated)
    loss = Ladv + alpha * Lcontent
    loss.backward()

    # 가중치 업데이트
    optimizer.step()

    return loss

In [ ]:
import tqdm

num_epochs = 10
alpha = 1

for epoch in tqdm.trange(num_epochs):
    for hate_input_ids, clean_input_ids in combined_data_loader:

        generated_text = model.generate(input_ids=hate_input_ids)

        # Discriminator 훈련
        discriminator_loss = train_discriminator(Discriminator, optimizerD, clean_input_ids, generated_text)
        # Generator 훈련
        generator_loss = train_generator(model, Discriminator, optimizerG, hate_input_ids, generated_text, alpha)

        # 진행 상태 출력
        tqdm.write(f"Epoch [{epoch + 1}/{num_epochs}] - Generator Loss: {generator_loss}, Discriminator Loss: {discriminator_loss}")